In [1]:
import numpy as np
import os
from PIL import Image
from scipy import signal
from scipy import interpolate
import matplotlib.pyplot as plt
%matplotlib inline
# Local import
from dataprocess import DataLoader, Blurr, Add_noise
from estimator import Estimator_Lap
from posttreat import Display
from posttreat import Error_Display

##### Kernel

In [2]:
M         = 20
Nx, Ny    = np.meshgrid(np.linspace(-1,1,2*M), np.linspace(-1,1,2*M))
d         = np.sqrt(Nx*Nx+Ny*Ny)
sigma, mu = 0.1, 0.0
K         = np.exp(-( (d-mu)**2 / ( 2.0 * sigma**2 ) ) )
K         = K/K.sum()

##### Image

In [3]:
file_name = 'Images'
im_name   = 'image05.jpg'
x_i       = DataLoader(file_name,im_name)
# Taille de l'image
print("Taille de l'image :")
x_i.shape

Taille de l'image :


(321, 481)

In [4]:
x_i  = x_i[:-1,:-1]
x_b  = Blurr(x_i,K)
x_o1 = Add_noise(x_b)
x_o2 = Add_noise(x_b, noise_level = 0.1)

##### Reconstruction

In [5]:
alpha = 1
mu    = 1
x_r   = Estimator_Lap(M,x_i,x_b,alpha,mu,niter=500)

ValueError: operands could not be broadcast together with shapes (480,320) (320,480) 